# Getting started with Amazon Braket Hybrid Jobs

This tutorial shows how to run your first Amazon Braket Hybrid Job. To get started, we consider small circuits with only one qubit and one gate.


## Learning outcomes 
* Write an algorithm script to run on Braket Jobs
* Understand how to run scripts or functions
* Create a job on Braket simulators or QPUs
* Monitoring the job state
* Save results from a job
* Using a specific AWS session
* Running jobs with priority on QPUs
* Use local jobs to quickly test and debug scripts
* Create a Braket Job using the Braket console

## Writing an algorithm script

To create a Braket Job, you first need a Python script to run. In this example, it's contained in `algorithm_script.py`. The script is printed in the code cell below for convenience. 

As shown, each of the circuits has only one $X$ rotation gate with a random angle. The circuit is repeated five times with different random rotations. Note that the algorithm script does not specify the backend Amazon Braket device ARN explicitly. Instead, it is provided through environment variables such as `os.environ["AMZN_BRAKET_DEVICE_ARN"]` that are passed to the algorithm script when creating the job. 

In [1]:
%%script echo "This block is a copy of the algorithm script."

import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result


print("Test job started!")

# Use the device declared in the creation script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_circuit = Circuit().rx(0, angle)

    task = device.run(random_circuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list})

print("Test job completed!")

This block is a copy of the algorithm script.


## Creating your first job

Once the script is finalized, you can create a Braket Job with `AwsQuantumJob`. When the Braket Job is created, it starts the job instance (based on EC2) and spins up a Docker container. The instance type, container, and other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks to learn more about how to customize your jobs.

This example uses the following inputs for `AwsQuantumJob`:
- <b>device</b>: The ARN of the Braket simulator or QPU to use in the job. It will be passed as an environment variable to the algorithm script. 
- <b>source_module</b>: The path to a file or a Python module that contains your algorithm script. It will be uploaded to the container for running the Braket Job.
- <b>wait_until_complete (optional)</b>: If true, the function call will wait until the Braket Job is completed and will additionally print logs to the local console. Otherwise, it will run asynchronously. 

In [2]:
from braket.aws import AwsQuantumJob

In [4]:
# This cell should take about 5 mins
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    wait_until_complete=False,
)

In this example, the algorithm is defined by a single file, so the `source_module` is `algorithm_script.py`. Depending on your application, there are other options for setting the source module. For example, if you wish to only execute a part of `algorithm_script.py` at the start of a Braket Job, you can package that part to be a `starting_function()`. Then assign the function as the entry point by adding the `entry_point` input argument.

In [5]:
source_module = "algorithm_script.py"
entry_point = "algorithm_script:starting_function"

If your algorithm script requires other dependency such as helper functions, you can put them all in one folder, say the `algorithm_folder`. The input arguments would then be

In [6]:
source_module = "algorithm_folder"
entry_point = "algorithm_folder.algorithm_script:starting_function"

## Checking job state and loading results

The status of a Braket Job can be checked by calling `job.state()`. The state will be one of "QUEUED", "INITIALIZING", "RUNNING", "FAILED", or "COMPLETED". 

In [7]:
job.state()

'QUEUED'

Once completed, the result can be retrieved using `job.result()`. Logs and metadata are also accessible via `job.logs()` and `job.metadata()`. If you lose the reference to the job object, you can always reinstantiate it using your job ARN as `job=AwsQuantumJob("your-job-arn")`. The ARN of a job can be found in the Amazon Braket Console. By default the ARN of a job will be "`arn:aws:braket:<region>:<account_id>:job/<job_name>`". 

In [8]:
results = job.result()  # will return once job.state() = "COMPLETED", should be 6 minutes
print("counts: ", results["counts"])
print("angles: ", results["angles"])

counts:  [{'1': 99, '0': 1}, {'0': 99, '1': 1}, {'1': 92, '0': 8}, {'0': 97, '1': 3}, {'1': 36, '0': 64}]
angles:  [3.3051995719859417, -6.17347000615851, -2.5079863385646535, -0.28662366922144145, 1.3012932727218849]


In [14]:
# print(job.logs()) # uncomment to print logs

You can also download the result to a local directory.

In [10]:
job.download_result()  # download job result to local directory

## Running jobs with priority on QPUs

With Braket Jobs, you can run hybrid algorithms on all QPUs available through Amazon Braket. When you select a QPU as your device, your job will have priority access for the duration of your job. Quantum tasks created as part of your job will be executed ahead of other tasks in the device queue. This reduces the risk of certain tasks being delayed or drifting calibrations on the device. 

You can seamlessly swap the SV1 simulator for a QPU by changing the device argument in `AwsQuantumJob.create()`. When you create the job, Amazon Braket will wait for the QPU to become available before initializing the job. The following cell creates the job to run on Rigetti Aspen-9 QPU. You can check on the status in the console or by using `job.state()`.
 
Note that the Braket Job will automatically select the AWS region where the device is available. 

In [ ]:
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/qpu/rigetti/Aspen-9",
    source_module="algorithm_script.py",
    wait_until_complete=False,
)

In [11]:
job.state()

'COMPLETED'

## AWS Sessions

You can customize the default location where Braket Jobs saves and loads results in Amazon S3 by providing the AWS session information. The name of the S3 bucket needs to start with "amazon-braket-"., and it must be in the same region as the job being created

In [12]:
from braket.aws import AwsSession

# Set Amazon S3 bucket
aws_session = AwsSession(default_bucket="amazon-braket-bucket-name")

To create a Braket Job with this S3 bucket, you pass `aws_session` as an argument to ` AwsQuantumJob.create()`

```
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    aws_session=aws_session # using specific S3 bucket
)
```

## Debugging with local Braket Jobs

For faster testing and debugging of your code, you can run a job locally in your own environment. This feature requires Docker to be installed in your local environment. Amazon Braket Notebook Instances have Docker pre-installed, so you can test local jobs in hosted notebooks instantly. To install Docker in your local environment, follow these [instructions](https://docs.docker.com/get-docker/). When a local job is created for the first time, it will take longer because it needs to build the container. The subsequent runs will be faster. Note, that local jobs will not be visible in the Amazon Braket Console.

To run a job in the local mode, simply create a `LocalQuantumJob`instead of `AwsQuantumJob`. Local jobs always run synchronously and display the logs. 

In [13]:
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min
job = LocalQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
)

Using the short-lived AWS credentials found in session. They might expire while running.


Boto3 Version:  1.18.33
Beginning Setup
Getting setup script from  s3://braket-external-assets-preview-us-west-2/HybridJobsAccess/scripts/setup-container.sh
download: s3://braket-external-assets-preview-us-west-2/HybridJobsAccess/libraries/amazon-braket-schemas-python.tar.gz to ../../braket/additional_lib/original/amazon-braket-schemas-python.tar.gz
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.coveragerc
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/bug_report.md
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/config.yml
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/documentation_req

## Creating a Braket Job from the Braket console

Besides creating a Braket Job programmatically using `AwsQuantumJob.create`, there is also an option to create a job in the Braket console. Follow [this link](https://us-west-2.console.aws-dev.amazon.com/braket/home#/job/create) to the "Create job" page. First, we need to give our new job a unique name. By default, Amazon Braket leverages the AWSServiceRoleForAmazonBraket role for your job to ensure all the necessary permission to AWS resources exist during execution. The default S3 bucket, where all inputs and outputs for your job are saved follows the convention `amazon-braket-<region>-<account number>/jobs/<job-name>`. Amazon Braket will create a bucket on your behalf if it doesn't exist. You have the option to customize these defaults in the "Advanced settings" tab.

<img src="console_figures/1-create.png" alt="Create job. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

Next, you select the algorithm script for the job. The script can be uploaded directly in the console as a single file. If there are many files, such as helper functions or other dependencies that go with the algorithm script, you have the option to upload the files to a S3 bucket and provide the path of the bucket.
<img src="console_figures/2-algorithm.png" alt="Select algorithm script. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

Third, you select a container environment for your job. The "Base" container is enough for the example in this notebook. For information about using other pre-built or custom containers, see the QAOA and the BYOC example notebooks. 

<img src="console_figures/3-container.png" alt="Select a container. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

You must also select a Braket managed simulator or QPU for your job and configure the execution settings. Finally, there are optional hyperparameter and data inputs, as well as the possibility to customize the default locations for checkpoints and output data. We will dive deeper into these advanced use cases in other example notebooks. After finishing all the settings, you create the job by clicking the "create job" button. We can now view the progress in the Braket console.

<img src="console_figures/4-execution.png" alt="Configuration for execution. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

## Summary

In this tutorial, we have created our first Braket Job with a simple batch of five circuits using the Amazon Braket SDK and, as an alternative, from the Braket console. We learned how to change the Amazon S3 bucket and the AWS region for a job, and how to save results. We learned how to seamlessly change the device to run on simulators or QPUs. We used local jobs to quickly test code. Finally, we created the same job using the Braket console.